In [ ]:
import tensorflow_datasets as tfds
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Layer
from tensorflow.keras.layers import (GlobalAveragePooling2D, Activation, MaxPooling2D, Add, Conv2D, MaxPool2D, Dense,
                                     Flatten, InputLayer, BatchNormalization, Input, Embedding, Permute,
                                     Dropout, RandomFlip, RandomRotation, LayerNormalization, MultiHeadAttention,
                                     RandomContrast, Rescaling, Resizing, Reshape, LeakyReLU)
from tensorflow.keras.losses import BinaryCrossentropy,CategoricalCrossentropy, SparseCategoricalCrossentropy
from tensorflow.keras.metrics import Accuracy,TopKCategoricalAccuracy, CategoricalAccuracy, SparseCategoricalAccuracy
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import (Callback, CSVLogger, EarlyStopping, LearningRateScheduler,
                                        ModelCheckpoint, ReduceLROnPlateau)
from tensorflow.keras.regularizers import L2, L1
from tensorflow.keras.initializers import RandomNormal

In [ ]:

!pip install -q kaggle
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 /root/.kaggle/kaggle.json
!kaggle datasets download -d huanghanchina/pascal-voc-2012

mkdir: cannot create directory ‘/root/.kaggle’: File exists
cp: cannot stat 'kaggle.json': No such file or directory
chmod: cannot access '/root/.kaggle/kaggle.json': No such file or directory
Dataset URL: https://www.kaggle.com/datasets/huanghanchina/pascal-voc-2012
License(s): DbCL-1.0
pascal-voc-2012.zip: Skipping, found more recently modified local copy (use --force to force download)


In [ ]:

!unzip "/content/pascal-voc-2012.zip" -d "/content/dataset/"

Streaming output truncated to the last 5000 lines.
  inflating: /content/dataset/voc2012/VOC2012/SegmentationClass/2008_001829.png  
  inflating: /content/dataset/voc2012/VOC2012/SegmentationClass/2008_001874.png  
  inflating: /content/dataset/voc2012/VOC2012/SegmentationClass/2008_001876.png  
  inflating: /content/dataset/voc2012/VOC2012/SegmentationClass/2008_001882.png  
  inflating: /content/dataset/voc2012/VOC2012/SegmentationClass/2008_001885.png  
  inflating: /content/dataset/voc2012/VOC2012/SegmentationClass/2008_001895.png  
  inflating: /content/dataset/voc2012/VOC2012/SegmentationClass/2008_001896.png  
  inflating: /content/dataset/voc2012/VOC2012/SegmentationClass/2008_001926.png  
  inflating: /content/dataset/voc2012/VOC2012/SegmentationClass/2008_001966.png  
  inflating: /content/dataset/voc2012/VOC2012/SegmentationClass/2008_001971.png  
  inflating: /content/dataset/voc2012/VOC2012/SegmentationClass/2008_001992.png  
  inflating: /content/dataset/voc2012/VOC2012/S

In [ ]:
val_list=['2007_000027.jpg','2007_000032.jpg','2007_000033.jpg','2007_000039.jpg','2007_000042.jpg','2007_000061.jpg',
          '2007_000063.jpg','2007_000068.jpg','2007_000121.jpg','2007_000123.jpg','2007_000129.jpg','2007_000170.jpg',
          '2007_000175.jpg','2007_000187.jpg','2007_000241.jpg','2007_000243.jpg','2007_000250.jpg','2007_000256.jpg',
          '2007_000272.jpg','2007_000323.jpg','2007_000332.jpg','2007_000333.jpg','2007_000346.jpg','2007_000363.jpg',
          '2007_000364.jpg','2007_000392.jpg','2007_000423.jpg','2007_000452.jpg','2007_000464.jpg','2007_000480.jpg',
          '2007_000491.jpg','2007_000504.jpg','2007_000515.jpg','2007_000528.jpg','2007_000529.jpg','2007_000549.jpg',
          '2007_000559.jpg','2007_000572.jpg','2007_000584.jpg','2007_000629.jpg','2007_000636.jpg','2007_000645.jpg',
          '2007_000648.jpg','2007_000661.jpg','2007_000663.jpg','2007_000664.jpg','2007_000676.jpg','2007_000713.jpg',
          '2007_000720.jpg','2007_000727.jpg','2007_000733.jpg','2007_000738.jpg','2007_000762.jpg','2007_000768.jpg',
          '2007_000783.jpg','2007_000793.jpg','2007_000799.jpg','2007_000804.jpg','2007_000807.jpg','2007_000822.jpg',
          '2007_001299.jpg','2007_001311.jpg','2007_001321.jpg','2007_001340.jpg']

In [ ]:
train_images='/content/dataset/VOC2012/JPEGImages/'
train_maps='/content/dataset/VOC2012/Annotations/'

val_images='/content/dataset/VOC2012/ValJPEGImages/'
val_maps='/content/dataset/VOC2012/ValAnnotations/'

classes=['aeroplane','bicycle','bird','boat','bottle','bus','car','cat','chair','cow','diningtable',
         'dog','horse','motorbike','person','pottedplant','sheep','sofa','train','tvmonitor']

B=2
n_classes=len(classes)
h,w =224,224
split_size=h//32
print(split_size)
n_epochs=135
N_CLASSES=n_classes
batch_size=32

7


In [ ]:
import shutil

!mkdir /content/dataset/VOC2012/ValJPEGImages/
!mkdir /content/dataset/VOC2012/ValAnnotations/

In [ ]:
for name in val_list:
  shutil.move(train_maps+name[:-3]+"xml", val_maps+name[:-3]+"xml")

In [ ]:
for name in val_list:
  shutil.move(train_images+name, val_images+name)

In [ ]:
import tensorflow as tf

In [ ]:
import xml.etree.ElementTree as ET
def preprocess(filename):
  tree=ET.parse(filename)
  root=tree.getroot()
  size_tree=root.find('size')
  width=int(size_tree.find('width').text)
  height=int(size_tree.find('height').text)
  bounding_boxes=[]
  for objtree in  root.findall('object'):
    for img in objtree.findall('bndbox'):
      xmin=int(img.find('xmin').text)
      ymin=int(img.find('ymin').text)
      xmax=int(img.find('xmax').text)
      ymax=int(img.find('ymax').text)
      break
    class_name=objtree.find('name').text
    class_id=classes.index(class_name)
    class_dict={classes[i]:i for i in range(len(classes))}

    x=(xmin+xmax)/(2.0*width)
    y=(ymin+ymax)/(2.0*height)
    w=(xmax-xmin)/width
    h=(ymax-ymin)/height
    bounding_box=[x,y,w,h,class_dict[class_name]]
    bounding_boxes.append(bounding_box)

  return tf.convert_to_tensor(bounding_boxes,tf.float32)



In [ ]:
def generate_output(bounding_boxes):
  output_label=np.zeros((split_size,split_size,n_classes))
  for b in range(len(bounding_boxes)):
    x,y=bounding_boxes[...,b,:2]*split_size
    x,y=int(x),int(y)
    output_label[x,y,0:5]=[1.,x%1,y%1,bounding_boxes[...,b,2],bounding_boxes[...,b,3]]
    output_label[x,y,int(bounding_boxes[...,b,4])]=1

  return tf.convert_to_tensor(output_label,tf.float32)


In [ ]:
os.listdir(train_maps)

['2010_000750.xml',
 '2012_000637.xml',
 '2012_003303.xml',
 '2008_004235.xml',
 '2011_004284.xml',
 '2010_003375.xml',
 '2008_000415.xml',
 '2009_003168.xml',
 '2010_001480.xml',
 '2008_008048.xml',
 '2010_002107.xml',
 '2010_001787.xml',
 '2011_003997.xml',
 '2011_000054.xml',
 '2011_001066.xml',
 '2011_004720.xml',
 '2011_004032.xml',
 '2010_002469.xml',
 '2008_000103.xml',
 '2008_001653.xml',
 '2011_006998.xml',
 '2009_002169.xml',
 '2012_002060.xml',
 '2008_005611.xml',
 '2008_005279.xml',
 '2010_005002.xml',
 '2010_004642.xml',
 '2011_000861.xml',
 '2011_000698.xml',
 '2011_002173.xml',
 '2009_002009.xml',
 '2010_003954.xml',
 '2011_004022.xml',
 '2009_002632.xml',
 '2008_001098.xml',
 '2009_001801.xml',
 '2011_002330.xml',
 '2008_002412.xml',
 '2010_001669.xml',
 '2008_003802.xml',
 '2011_002312.xml',
 '2009_002326.xml',
 '2010_001241.xml',
 '2010_005601.xml',
 '2010_003514.xml',
 '2009_005098.xml',
 '2012_002679.xml',
 '2011_001127.xml',
 '2008_002536.xml',
 '2009_004895.xml',


In [ ]:
im_paths=[]
xml_paths=[]
val_im_paths=[]
val_xml_paths=[]

for name in os.listdir(train_maps):
  im_paths.append(train_images+name[:-3]+'jpg')
  xml_paths.append(train_maps+name)

for name in os.listdir(val_maps):
  val_im_paths.append(val_images+name[:-3]+'jpg')
  val_xml_paths.append(val_maps+name)







In [ ]:
path=val_xml_paths[0]
preprocess(path)

<tf.Tensor: shape=(2, 5), dtype=float32, numpy=
array([[0.38266668, 0.566     , 0.38666666, 0.124     , 9.        ],
       [0.39866668, 0.497     , 0.488     , 0.186     , 9.        ]],
      dtype=float32)>

In [ ]:
print(len(xml_paths),len(im_paths))
print(len(val_xml_paths),len(val_im_paths))

17061 17061
64 64


In [ ]:
def get_imbboxes(im_path,xml_path):
  img=tf.io.decode_jpeg(tf.io.read_file(im_path))
  img=tf.cast(tf.image.resize(img, [h,w]),dtype=tf.float32)

  bboxes=tf.numpy_function(func=preprocess, inp=[xml_path], Tout=tf.float32)
  return img,bboxes

In [ ]:
train_data=tf.data.Dataset.from_tensor_slices((im_paths,xml_paths))
val_data=tf.data.Dataset.from_tensor_slices((val_im_paths,val_xml_paths))

In [ ]:

train_dataset=train_data.map(get_imbboxes)
val_dataset=val_data.map(get_imbboxes)

In [ ]:
for img in val_data.take(1):
  print(img)

(<tf.Tensor: shape=(), dtype=string, numpy=b'/content/dataset/VOC2012/ValJPEGImages/2007_000464.jpg'>, <tf.Tensor: shape=(), dtype=string, numpy=b'/content/dataset/VOC2012/ValAnnotations/2007_000464.xml'>)


In [ ]:

for i,j in train_dataset.skip(2):
  print(i.shape,j)
  break

(224, 224, 3) tf.Tensor([[ 0.501       0.50133336  0.998       0.99733335 14.        ]], shape=(1, 5), dtype=float32)


In [ ]:
import albumentations as A
import cv2
transforms=A.Compose(
    [
        A.Resize(h,w),
        A.RandomCrop(
            width=np.random.randint(0.9*h,h),
            height=np.random.randint(0.9*w,w)

        ),
        A.RandomScale(0,0.1,cv2.INTER_LINEAR),
        A.HorizontalFlip(p=0.5),
        A.VerticalFlip(p=0.5),


    ]
)

In [ ]:
bbox_params=A.BboxParams(format='yolo', )

In [ ]:

def aug_albument(image,bboxes):
  augmented=transforms(image=image,bboxes=bboxes)
  return [tf.convert_to_tensor(augmented["image"],dtype=tf.float32),
          tf.convert_to_tensor(augmented["bboxes"],dtype=tf.float32)]

In [ ]:

def process_data(image,bboxes):
    aug= tf.numpy_function(func=aug_albument, inp=[image,bboxes], Tout=(tf.float32,tf.float32))
    return aug[0],aug[1]

In [ ]:
def preprocess_augment(img,y):
  img = tf.image.random_brightness(img, max_delta=50.)
  img = tf.image.random_saturation(img, lower=0.5, upper=1.5)
  img = tf.image.random_contrast(img, lower=0.5, upper=1.5)
  #img = tf.image.random_hue(img, max_delta=0.5 )
  img = tf.clip_by_value(img, 0, 255)
  labels=tf.numpy_function(func=generate_output, inp=[y], Tout=(tf.float32))
  return img,labels

In [ ]:

def preprocess(img,y):
  img = tf.cast(tf.image.resize(img, size=[H, W]), dtype=tf.float32)

  labels=tf.numpy_function(func=generate_output, inp=[y], Tout=(tf.float32))
  return img,labels

In [ ]:

train_dataset=train_dataset.map(preprocess_augment)
val_dataset=val_dataset.map(preprocess)

In [ ]:
for img,xml_img in train_dataset.take(1):
  print(img.shape,xml_img.shape)
  break

(224, 224, 3) (7, 7, 20)


In [ ]:
train_dataset=(
  train_dataset.
  batch(32).
  prefetch(tf.data.AUTOTUNE)
)


val_dataset=(
  val_dataset.
  batch(32).
  prefetch(tf.data.AUTOTUNE)
)

In [ ]:
for img,xml_img in train_dataset.take(1):
  print(img.shape,xml_img.shape)

(32, 224, 224, 3) (32, 7, 7, 20)


In [ ]:

for i,j in train_dataset.take(1):
  print(i.shape,j)
  break

(32, 224, 224, 3) tf.Tensor(
[[[[0. 0. 0. ... 0. 0. 0.]
   [0. 0. 0. ... 0. 0. 0.]
   [0. 0. 0. ... 0. 0. 0.]
   ...
   [0. 0. 0. ... 0. 0. 0.]
   [0. 0. 0. ... 0. 0. 0.]
   [0. 0. 0. ... 0. 0. 0.]]

  [[0. 0. 0. ... 0. 0. 0.]
   [0. 0. 0. ... 0. 0. 0.]
   [0. 0. 0. ... 0. 0. 0.]
   ...
   [0. 0. 0. ... 0. 0. 0.]
   [0. 0. 0. ... 0. 0. 0.]
   [0. 0. 0. ... 0. 0. 0.]]

  [[0. 0. 0. ... 0. 0. 0.]
   [0. 0. 0. ... 0. 0. 0.]
   [0. 0. 0. ... 0. 0. 0.]
   ...
   [0. 0. 0. ... 0. 0. 0.]
   [0. 0. 0. ... 0. 0. 0.]
   [0. 0. 0. ... 0. 0. 0.]]

  ...

  [[0. 0. 0. ... 0. 0. 0.]
   [0. 0. 0. ... 0. 0. 0.]
   [0. 0. 0. ... 0. 0. 0.]
   ...
   [0. 0. 0. ... 0. 0. 0.]
   [0. 0. 0. ... 0. 0. 0.]
   [0. 0. 0. ... 0. 0. 0.]]

  [[0. 0. 0. ... 0. 0. 0.]
   [0. 0. 0. ... 0. 0. 0.]
   [0. 0. 0. ... 0. 0. 0.]
   ...
   [0. 0. 0. ... 0. 0. 0.]
   [0. 0. 0. ... 0. 0. 0.]
   [0. 0. 0. ... 0. 0. 0.]]

  [[0. 0. 0. ... 0. 0. 0.]
   [0. 0. 0. ... 0. 0. 0.]
   [0. 0. 0. ... 0. 0. 0.]
   ...
   [0. 0. 0. ... 0. 0

In [ ]:

for name in val_list:
  shutil.move(train_maps+name[:-3]+"xml", val_maps+name[:-3]+"xml")

['2011_006315.jpg',
 '2012_003967.jpg',
 '2008_003827.jpg',
 '2009_003694.jpg',
 '2010_006852.jpg',
 '2011_001952.jpg',
 '2011_003664.jpg',
 '2010_000050.jpg',
 '2008_008309.jpg',
 '2008_004616.jpg',
 '2009_004813.jpg',
 '2009_002358.jpg',
 '2009_003284.jpg',
 '2011_002181.jpg',
 '2008_005832.jpg',
 '2008_006178.jpg',
 '2007_004241.jpg',
 '2008_001263.jpg',
 '2008_008517.jpg',
 '2008_001649.jpg',
 '2008_008685.jpg',
 '2010_005997.jpg',
 '2012_003937.jpg',
 '2008_001885.jpg',
 '2010_005709.jpg',
 '2012_003108.jpg',
 '2009_004670.jpg',
 '2011_006819.jpg',
 '2011_004976.jpg',
 '2009_002799.jpg',
 '2012_002308.jpg',
 '2010_000582.jpg',
 '2011_002436.jpg',
 '2011_003134.jpg',
 '2010_004629.jpg',
 '2011_005640.jpg',
 '2008_000002.jpg',
 '2011_003924.jpg',
 '2010_000738.jpg',
 '2011_004672.jpg',
 '2010_003458.jpg',
 '2008_007389.jpg',
 '2008_006034.jpg',
 '2008_008480.jpg',
 '2011_006193.jpg',
 '2008_003110.jpg',
 '2011_001974.jpg',
 '2008_000953.jpg',
 '2012_003093.jpg',
 '2012_001735.jpg',


In [ ]:
B=2
NUM_FILTERS=512
OUTPUT_DIM=N_CLASSES+5*B
SPLIT_SIZE=split_size




In [ ]:
base_model=tf.keras.applications.efficientnet.EfficientNetB1(
    weights='imagenet',
    input_shape=(H,W,3),
    include_top=False,
)
base_model.trainable=False

27018416/27018416 [==============================] - 0s 0us/step


In [ ]:
model=tf.keras.Sequential([
  base_model,
  Conv2D(NUM_FILTERS,(3,3), padding = 'same',kernel_initializer='he_normal',),
  BatchNormalization(),
  LeakyReLU(alpha=0.1),

  Conv2D(NUM_FILTERS,(3,3),padding = 'same',kernel_initializer='he_normal',),
  BatchNormalization(),
  LeakyReLU(alpha=0.1),

  Conv2D(NUM_FILTERS,(3,3),padding = 'same',kernel_initializer='he_normal',),
  BatchNormalization(),
  LeakyReLU(alpha=0.1),

  Conv2D(NUM_FILTERS,(3,3),padding = 'same',kernel_initializer='he_normal',),
  LeakyReLU(alpha=0.1),

  Flatten(),

  Dense(NUM_FILTERS,kernel_initializer='he_normal',),
  BatchNormalization(),
  LeakyReLU(alpha=0.1),

  Dropout(0.5),

  Dense(SPLIT_SIZE*SPLIT_SIZE*OUTPUT_DIM,activation='sigmoid'),

  Reshape((SPLIT_SIZE,SPLIT_SIZE,OUTPUT_DIM)),
])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 efficientnetb1 (Functional  (None, 7, 7, 1280)        6575239   
 )                                                               
                                                                 
 conv2d_4 (Conv2D)           (None, 7, 7, 512)         5898752   
                                                                 
 batch_normalization_4 (Bat  (None, 7, 7, 512)         2048      
 chNormalization)                                                
                                                                 
 leaky_re_lu_5 (LeakyReLU)   (None, 7, 7, 512)         0         
                                                                 
 conv2d_5 (Conv2D)           (None, 7, 7, 512)         2359808   
                                                                 
 batch_normalization_5 (Bat  (None, 7, 7, 512)         2

In [ ]:

def compute_iou(boxes1, boxes2):
    boxes1_t = tf.stack([boxes1[..., 0] - boxes1[..., 2] / 2.0,
                         boxes1[..., 1] - boxes1[..., 3] / 2.0,
                         boxes1[..., 0] + boxes1[..., 2] / 2.0,
                         boxes1[..., 1] + boxes1[..., 3] / 2.0],
                        axis=-1)

    boxes2_t = tf.stack([boxes2[..., 0] - boxes2[..., 2] / 2.0,
                         boxes2[..., 1] - boxes2[..., 3] / 2.0,
                         boxes2[..., 0] + boxes2[..., 2] / 2.0,
                         boxes2[..., 1] + boxes2[..., 3] / 2.0],
                        axis=-1)
    lu = tf.maximum(boxes1_t[..., :2], boxes2_t[..., :2])
    rd = tf.minimum(boxes1_t[..., 2:], boxes2_t[..., 2:])

    intersection = tf.maximum(0.0, rd - lu)
    inter_square = intersection[..., 0] * intersection[..., 1]

    square1 = boxes1[..., 2] * boxes1[..., 3]
    square2 = boxes2[..., 2] * boxes2[..., 3]

    union_square = tf.maximum(square1 + square2 - inter_square, 1e-10)
    return tf.clip_by_value(inter_square / union_square, 0.0, 1.0)


In [ ]:

def difference(x,y):
  return tf.reduce_sum(tf.square(y-x))

In [ ]:
def yolo_loss(y_true, y_pred):
  target = y_true[...,0]

  ###################### OBject Loss
  y_pred_extract = tf.gather_nd(y_pred, tf.where(target[:]==1))
  y_target_extract = tf.gather_nd(y_true, tf.where(target[:]==1))

  rescaler = tf.where(target[:]==1)*SPLIT_SIZE
  upscaler_1 = tf.concat([rescaler[:,1:],tf.zeros([len(rescaler),2], dtype=tf.int64)],axis=-1)

  target_upscaler_2 = tf.repeat([[float(SPLIT_SIZE),float(SPLIT_SIZE),H,W]],
                       repeats=[len(rescaler)], axis=0)*tf.cast(y_target_extract[...,1:5], dtype = tf.float32)
  pred_1_upscaler_2 = tf.repeat([[float(SPLIT_SIZE),float(SPLIT_SIZE),H,W]],
                      repeats=[len(rescaler)], axis=0)*tf.cast(y_pred_extract[...,1:5], dtype = tf.float32)
  pred_2_upscaler_2 = tf.repeat([[float(SPLIT_SIZE),float(SPLIT_SIZE),H,W]],
                      repeats=[len(rescaler)], axis=0)*tf.cast(y_pred_extract[...,6:10], dtype = tf.float32)

  target_orig = tf.cast(upscaler_1, dtype = tf.float32)+target_upscaler_2
  pred_1_orig = tf.cast(upscaler_1, dtype = tf.float32)+pred_1_upscaler_2
  pred_2_orig = tf.cast(upscaler_1, dtype = tf.float32)+pred_2_upscaler_2

  mask =tf.cast(tf.math.greater(compute_iou(target_orig,pred_2_orig),
                                         compute_iou(target_orig,pred_1_orig)),dtype=tf.int32)

  y_pred_joined=tf.transpose(tf.concat([tf.expand_dims(y_pred_extract[...,0],axis=0),
                        tf.expand_dims(y_pred_extract[...,5],axis=0)],axis=0))

  obj_pred = tf.gather_nd(y_pred_joined,tf.stack([tf.range(len(rescaler)),mask],axis=-1))

  object_loss = difference(tf.cast(obj_pred,dtype =tf.float32)
                            ,tf.cast(tf.ones([len(rescaler)]),dtype=tf.float32))

  ####################### For No object
  y_pred_extract = tf.gather_nd(y_pred[...,0:B*5], tf.where(target[:]==0))
  y_target_extract = tf.zeros(len(y_pred_extract))

  no_object_loss_1 = difference(tf.cast(y_pred_extract[...,0],dtype =tf.float32)
                            ,tf.cast(y_target_extract,dtype=tf.float32))

  no_object_loss_2 = difference(tf.cast(y_pred_extract[...,5],dtype =tf.float32)
                            ,tf.cast(y_target_extract,dtype=tf.float32))

  no_object_loss = no_object_loss_1+no_object_loss_2

  ######################## For OBject class loss
  y_pred_extract = tf.gather_nd(y_pred[...,10:],tf.where(target[:]==1))
  class_extract = tf.gather_nd(y_true[...,5:],tf.where(target[:]==1))

  class_loss = difference(tf.cast(y_pred_extract,dtype =tf.float32)
                                ,tf.cast(class_extract,dtype=tf.float32))

  ######################### For object bounding box loss
  y_pred_extract = tf.gather_nd(y_pred[...,0:B*5], tf.where(target[:]==1))
  centre_joined=tf.stack([y_pred_extract[...,1:3],y_pred_extract[...,6:8]],axis=1)
  centre_pred = tf.gather_nd(centre_joined,tf.stack([tf.range(len(rescaler)),mask],axis=-1))
  centre_target = tf.gather_nd(y_true[...,1:3], tf.where(target[:]==1))

  centre_loss = difference(centre_pred,centre_target)

  size_joined=tf.stack([y_pred_extract[...,3:5],y_pred_extract[...,8:10]],axis=1)

  size_pred = tf.gather_nd(size_joined,tf.stack([tf.range(len(rescaler)),mask],axis=-1))
  size_target = tf.gather_nd(y_true[...,3:5], tf.where(target[:]==1))

  size_loss = difference(tf.math.sqrt(tf.math.abs(size_pred)),tf.math.sqrt(tf.math.abs(size_target)))
  box_loss = centre_loss+size_loss

  lambda_coord = 5.0
  lambda_no_obj = 0.5

  loss = object_loss + (lambda_no_obj*no_object_loss)+ tf.cast(lambda_coord*box_loss,dtype=tf.float32)+ tf.cast(class_loss,dtype=tf.float32)
  return loss

In [ ]:
checkpoint_filepath='/content/drive/MyDrive/Bang/yolo_efficientnet_b1_new.h5'
callback = tf.keras.callbacks.ModelCheckpoint(
    filepath = checkpoint_filepath,
    save_weights_only=True,
    monitor='val_loss',
    mode='min',
    save_best_only=True
)

In [ ]:
def scheduler(epoch, lr):
  if epoch < 40:
    return 1e-3
  elif epoch>=40 and epoch<80:
    return 5e-4
  else:
    return 1e-4

In [ ]:
lr_callback = tf.keras.callbacks.LearningRateScheduler(scheduler)

In [ ]:
model.compile(
  loss=yolo_loss,
  optimizer=Adam(1e-3),
)

In [ ]:

history = model.fit(
  train_dataset,
  validation_data=val_dataset,
  verbose=1,
  epochs=10,
  callbacks = [lr_callback,callback]
)

Epoch 1/10


InvalidArgumentError: Graph execution error:

Detected at node gradient_tape/yolo_loss/sub_16/BroadcastGradientArgs defined at (most recent call last):
  File "/usr/lib/python3.10/runpy.py", line 196, in _run_module_as_main

  File "/usr/lib/python3.10/runpy.py", line 86, in _run_code

  File "/usr/local/lib/python3.10/dist-packages/colab_kernel_launcher.py", line 37, in <module>

  File "/usr/local/lib/python3.10/dist-packages/traitlets/config/application.py", line 992, in launch_instance

  File "/usr/local/lib/python3.10/dist-packages/ipykernel/kernelapp.py", line 619, in start

  File "/usr/local/lib/python3.10/dist-packages/tornado/platform/asyncio.py", line 195, in start

  File "/usr/lib/python3.10/asyncio/base_events.py", line 603, in run_forever

  File "/usr/lib/python3.10/asyncio/base_events.py", line 1909, in _run_once

  File "/usr/lib/python3.10/asyncio/events.py", line 80, in _run

  File "/usr/local/lib/python3.10/dist-packages/tornado/ioloop.py", line 685, in <lambda>
